In [92]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [93]:
from pyspark.sql import SparkSession

In [94]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

### 1. Connect Transactions and Merchants AS `sdf`

In [95]:
print("================ Transactions ================")
# transactions_sdf = read_file(spark, 'transactions_20210228_20210827_snapshot')
transactions_sdf = read_file(spark, 'transactions_20210828_20220227_snapshot')
# transactions_sdf = read_file(spark, 'transactions_20220228_20220828_snapshot')
temp = transactions_sdf.dropna(how='any').count()
print( f'There are {transactions_sdf.count()} data, and {temp} of them have no null value.')

================ Transactions ================
|> Loading File...
|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 14935                                
 merchant_abn   | 79417999332                          
 dollar_value   | 136.06570809815838                   
 order_id       | 23acbb7b-cf98-4580-9775-86b8e0a2bd88 
 order_datetime | 2021-11-26                           
only showing top 1 row



There are 4508106 data, and 4508106 of them have no null value.


In [96]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'merchants_data.parquet', '../data/curated/', truncate=20)
temp = merchants_sdf.dropna(how='any').count()
print( f'There are {merchants_sdf.count()} data, and {temp} of them have no null value.')

================ Merchant ================
|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------
 merchant_name | Felis Limited        
 tags          | furniture, home f... 
 merchant_abn  | 10023283211          
 take_rate     | 0.18                 
 type          | e                    
only showing top 1 row

There are 4026 data, and 4026 of them have no null value.


- Known that Transactions and Merchants table have no null value

In [97]:
sdf = transactions_sdf.join(merchants_sdf, how='left', on='merchant_abn')
temp = sdf.dropna(how='any', subset=['merchant_name']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 4508106 data, and 4323692 of them have no null value.


- After connect the Transactions and Merchants Table by `merchant_abn`, we find that some Transactions not from the merchants in the table. Because they are not in our list of rank therefore, delete those null value.

In [98]:
# sdf = sdf.dropna(how='any', subset=['merchant_name'])

- There are 4026 merchants, and for `transactions_20210228_20210827_snapshot` have 3643266 records. 

    After combination, there are **3494038** rest. **3643266 - 3494038 = 149228** records has been deleted. 
    
    `149228 / 3643266 = 0.04095995186736297`

In [99]:
sdf.show(5)

+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
|merchant_abn|user_id|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
| 79417999332|  14935|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|
| 46451548968|      1| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|Tempus Eu Ligula ...|health and beauty...|     6.04|   a|
| 89518629617|  14936|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|Vulputate Velit E...|tent and awning s...|     3.09|   c|
| 49167531725|      1| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|     Felis Institute|digital goods: bo...|     6.42|   a|
| 31101120643|  14936|25.228114942417797|8e301c0f-06ab-

### 2. `sdf` join with Customer

In [100]:
print("================ Customer ================")
customer_sdf = read_file(spark, 'new_consumer_data.parquet', '../data/curated/')
temp = customer_sdf.dropna(how='any').count()
print( f'There are {customer_sdf.count()} data, and {temp} of them have no null value.')

================ Customer ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------------------
 consumer_id | 680810                     
 name        | Courtney Mendez            
 address     | 802 Knapp Harbors Apt. 769 
 state       | QLD                        
 postcode    | 9013                       
 gender      | Female                     
 user_id     | 71144                      
only showing top 1 row

There are 499999 data, and 499999 of them have no null value.


In [101]:
sdf = sdf.join(customer_sdf, how='left', on='user_id')
temp = sdf.dropna(how='any', subset=['consumer_id']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 4508106 data, and 4508106 of them have no null value.


In [102]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+---------------+--------------------+-----+--------+------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|consumer_id|           name|             address|state|postcode|gender|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+---------------+--------------------+-----+--------+------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|    1059280|  Cameron Adams|0280 Carpenter Lodge|  QLD|    4563|  Male|
|  14935| 72472909171|52.973387548612614|01d968a6-dde1-48c...|    2021-12-01|   Nullam Consulting|digital goods: bo...|     6.33|   a|    1059280|  Cameron Adams|0280 Carpenter

### 3. `sdf` join with the external dataset  --- rrm

In [103]:
# print("================ RRM_External ================")
# rrm_sdf = read_file(spark, 'external_rrm.parquet', '../data/curated/', truncate=20)
# temp = rrm_sdf.dropna(how='any').count()
# print( f'There are {rrm_sdf.count()} data, and {temp} of them have no null value.')

In [104]:
# sdf = sdf.join(rrm_sdf, sdf.order_datetime == rrm_sdf.Date, 'left')
# temp = sdf.dropna(how='any', subset=['Date']).count()
# print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

In [105]:
# sdf.show(5)

In [106]:
# sdf = sdf.drop('Date')

- 处理 external table 的信息，只保留其所对应的洲的数值

In [107]:
# import pandas as pd

In [108]:
# cols = ['state', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT']
# temp_df = sdf.select(['order_id'] + cols).toDF('order_id', 'state', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT')
# temp_df = pd.DataFrame(temp_df.collect(), columns=['order_id'] + cols)

In [109]:
# temp_df['target'] = temp_df.apply(lambda x: x[x['state']], axis=1)
# temp_df = temp_df.drop(cols, axis=1)

In [110]:
# temp_sdf = spark.createDataFrame(temp_df)
# temp_sdf.show(5)

In [111]:
# sdf = sdf.join(temp_sdf, on='order_id', how='left')

In [112]:
# sdf.show(5)

### 4. `sdf` join with the external dataset  --- ncd

In [113]:
print("================  ================")
ncd_sdf = read_file(spark, 'external_ncd.parquet', '../data/curated/')
temp = ncd_sdf.dropna(how='any').count()
print( f'There are {ncd_sdf.count()} data, and {temp} have no null value.')

================  ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------
 Date            | 2020-01-25 
 New cases / day | 4          
only showing top 1 row

There are 959 data, and 959 have no null value.


In [114]:
ncd_sdf.show(5)

+----------+---------------+
|      Date|New cases / day|
+----------+---------------+
|2020-01-25|              4|
|2020-01-26|              0|
|2020-01-27|              1|
|2020-01-28|              0|
|2020-01-29|              4|
+----------+---------------+
only showing top 5 rows



In [115]:
sdf = sdf.join(ncd_sdf, sdf.order_datetime == ncd_sdf.Date, 'left')
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+---------------+--------------------+-----+--------+------+----------+---------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|consumer_id|           name|             address|state|postcode|gender|      Date|New cases / day|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+---------------+--------------------+-----+--------+------+----------+---------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|    1059280|  Cameron Adams|0280 Carpenter Lodge|  QLD|    4563|  Male|2021-11-26|          1,621|
|  14935| 72472909171|52.973387548612614|01d968a6-dde1-48c...|    20

In [116]:
temp = sdf.dropna(how='any', subset=['Date']).count()
print( f'There are {sdf.count()} data, and {temp} have no null value.')

There are 4508106 data, and 4508106 have no null value.


In [117]:
sdf = sdf.drop('Date')

In [118]:
# temp = sdf.to_pandas_on_spark()
# temp[temp.Date.isna()]

In [119]:
path = '../data/curated/data'
if (create_folder(path)):
    sdf.write.partitionBy('order_datetime').parquet(path, mode='append')
else:
    sdf.write.partitionBy('order_datetime').parquet(path, mode='overwrite')


|> The folder already exist!
|> Files already exist under this folder:
   ['order_datetime=2021-10-19', 'order_datetime=2021-10-26', 'order_datetime=2021-10-21', 'order_datetime=2021-10-28', 'order_datetime=2021-10-17', 'order_datetime=2021-10-10', 'order_datetime=2022-02-24', 'order_datetime=2022-01-03', 'order_datetime=2022-01-04', 'order_datetime=2022-02-23', 'order_datetime=2022-02-15', 'order_datetime=2022-02-12', 'order_datetime=2021-10-11', 'order_datetime=2021-10-29', 'order_datetime=2021-10-16', 'order_datetime=2021-10-20', 'order_datetime=2021-10-18', 'order_datetime=2021-10-27', 'order_datetime=2022-02-13', 'order_datetime=2022-02-14', 'order_datetime=2022-02-22', 'order_datetime=2022-01-05', 'order_datetime=2022-01-02', 'order_datetime=2022-02-25', 'order_datetime=2021-09-25', 'order_datetime=2021-12-09', 'order_datetime=2021-11-11', 'order_datetime=2021-11-29', 'order_datetime=2021-09-22', 'order_datetime=2021-11-16', 'order_datetime=2021-12-31', 'order_datetime=2021-09-14

### 5. `sdf` join with the fraud datasets

- consumer fraud datasets

In [120]:
sdf = read_file(spark, 'data', '../data/curated/')

|> Loading File...


|> Loading Finished!
-RECORD 0-----------------------------------------------
 user_id         | 1                                    
 merchant_abn    | 46451548968                          
 dollar_value    | 72.61581642788431                    
 order_id        | 76bab304-fa2d-4004-8179-8638b56a873e 
 merchant_name   | Tempus Eu Ligula Limited             
 tags            | health and beauty spas               
 take_rate       | 6.04                                 
 type            | a                                    
 consumer_id     | 1195503                              
 name            | Yolanda Williams                     
 address         | 413 Haney Gardens Apt. 742           
 state           | WA                                   
 postcode        | 6935                                 
 gender          | Female                               
 New cases / day | 1,621                                
 order_datetime  | 2021-11-26                           
only showi

In [121]:
consumer_fraud_prob = read_file(spark, file_name='consumer_fraud_probability.csv', type='csv')
temp1 = consumer_fraud_prob.dropna(how='any').count()

consumer_fraud_prob = (consumer_fraud_prob
 
 
 .withColumnRenamed('order_datetime','date')
 
 
 .withColumnRenamed('user_id', 'id'))
print( f'There are {consumer_fraud_prob.count()} data, and {temp1} have no null value.')

|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------
 user_id           | 6228             
 order_datetime    | 2021-12-19       
 fraud_probability | 97.6298077657765 
only showing top 1 row

There are 34864 data, and 34864 have no null value.


In [122]:
consumer_fraud_prob.show(5)

+-----+----------+-----------------+
|   id|      date|fraud_probability|
+-----+----------+-----------------+
| 6228|2021-12-19| 97.6298077657765|
|21419|2021-12-10|99.24738020302328|
| 5606|2021-10-17|84.05825045251777|
| 3101|2021-04-17|91.42192091901347|
|22239|2021-10-19|94.70342477508035|
+-----+----------+-----------------+
only showing top 5 rows



In [123]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------------+--------------------+-----+--------+------+---------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|consumer_id|             name|             address|state|postcode|gender|New cases / day|order_datetime|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------------+--------------------+-----+--------+------+---------------+--------------+
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|Tempus Eu Ligula ...|health and beauty...|     6.04|   a|    1195503| Yolanda Williams|413 Haney Gardens...|   WA|    6935|Female|          1,621|    2021-11-26|
|      1| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|     Felis Institute|digital goods: bo...|

In [124]:
sdf = sdf.join(consumer_fraud_prob, [sdf.user_id == consumer_fraud_prob.id, sdf.order_datetime == consumer_fraud_prob.date], how = 'inner')

In [125]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------+--------------------+-----+--------+------+---------------+--------------+---+----------+------------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|consumer_id|       name|             address|state|postcode|gender|New cases / day|order_datetime| id|      date| fraud_probability|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------+--------------------+-----+--------+------+---------------+--------------+---+----------+------------------+
|    226| 80682333501|15.725222523439964|1ec01b59-6a14-429...|          Orci Corp.|florists supplies...|     4.88|   b|     710483|Emily Singh|0107 Flores Stree...|   SA|    5642|Female|          1,621|    2021-11-26|226|2021-11-26|13.934739454

In [126]:
sdf = sdf.drop('id','date')

In [127]:
sdf.count()

70714

In [128]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------+--------------------+-----+--------+------+---------------+--------------+------------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|consumer_id|       name|             address|state|postcode|gender|New cases / day|order_datetime| fraud_probability|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+-----------+--------------------+-----+--------+------+---------------+--------------+------------------+
|    226| 80682333501|15.725222523439964|1ec01b59-6a14-429...|          Orci Corp.|florists supplies...|     4.88|   b|     710483|Emily Singh|0107 Flores Stree...|   SA|    5642|Female|          1,621|    2021-11-26|13.934739454102626|
|    226| 92773401740|3560.1210099528566|58860383-3f

In [137]:
temp_record_sdf(sdf, path = '../data/curated/consumer_fraud_table.parquet')

|> Waitting for saving...


|> Save Successfully!


- merchants fraud dataset

In [130]:
merchants_fraud_prob = read_file(spark, file_name='merchant_fraud_probability.csv', type='csv')
temp = merchants_fraud_prob.dropna(how='any').count()
merchants_fraud_prob = (merchants_fraud_prob
 
 
 .withColumnRenamed('order_datetime','date')
 
 
 .withColumnRenamed('merchant_abn', 'abn'))
print( f'There are {merchants_fraud_prob.count()} data, and {temp} have no null value.')

|> Loading File...
|> Loading Finished!
-RECORD 0-------------------------------
 merchant_abn      | 19492220327        
 order_datetime    | 2021-11-28         
 fraud_probability | 44.403658647495355 
only showing top 1 row

There are 114 data, and 114 have no null value.


In [131]:
merchants_fraud_prob.show(5)

+-----------+----------+------------------+
|        abn|      date| fraud_probability|
+-----------+----------+------------------+
|19492220327|2021-11-28|44.403658647495355|
|31334588839|2021-10-02| 42.75530083865367|
|19492220327|2021-12-22|38.867790051131095|
|82999039227|2021-12-19|  94.1347004808891|
|90918180829|2021-09-02| 43.32551731714902|
+-----------+----------+------------------+
only showing top 5 rows



In [132]:
sdf = sdf.join(merchants_fraud_prob, [sdf.order_datetime == merchants_fraud_prob.date, sdf.merchant_abn == merchants_fraud_prob.abn], 'inner')

In [133]:
sdf.count()

4055

In [134]:
sdf = sdf.drop('date', 'abn')

In [135]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+--------------+--------------------+-----+--------+-----------+---------------+--------------+------------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|consumer_id|          name|             address|state|postcode|     gender|New cases / day|order_datetime| fraud_probability|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+-----------+--------------+--------------------+-----+--------+-----------+---------------+--------------+------------------+
|     17| 76767266140|193.80299344578378|95330094-01e5-487...|Phasellus At Limited|furniture, home f...|     4.65|   b|    1331093| Alyssa Wilson|  44353 Nathan Ridge|  VIC|    3719|     Female|          1,621|    2021-11-26|29.539733207182703|
|    242| 4853464962

In [136]:
temp_record_sdf(sdf, path = '../data/curated/merchants_fraud_table.parquet')

|> Waitting for saving...


|> Save Successfully!
